In [0]:
from fastai.tabular import *


In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/Sapna20/ai3/master/train_file.csv')

In [37]:
df.head()

,ID,UsageClass,CheckoutType,CheckoutYear,CheckoutMonth,Checkouts,Title,Creator,Subjects,Publisher,PublicationYear,MaterialType
0,1,Physical,Horizon,2005,4,1,Tidal wave,NaN,"Tsunamis, Tsunamis Juvenile literature",NaN,NaN,BOOK
1,2,Physical,Horizon,2005,4,1,London holiday / Richard Peck.,"Peck, Richard, 1934-",NaN,"Viking,",1998.,BOOK
2,3,Physical,Horizon,2005,4,3,Cinco de Mayo : celebrating Hispanic pride / C...,"Gnojewski, Carol",Cinco de Mayo Mexican holiday History Juvenile...,"Enslow Publishers,",c2002.,BOOK
3,4,Physical,Horizon,2005,4,1,Annapolis,NaN,"War stories, Historical fiction, Domestic fict...",NaN,NaN,BOOK
4,5,Physical,Horizon,2005,4,1,As a man thinketh,NaN,Thought and thinking,NaN,NaN,BOOK


In [0]:
del df['ID']
del df['Title']
del df['Creator']

In [39]:
(df.set_index(df.columns.drop('Subjects',1).tolist())
       .Subjects.str.split(',', expand=True)
       .stack()
       .reset_index()
       .rename(columns={0:'Subjects'})
       .loc[:, df.columns]
)

,UsageClass,CheckoutType,CheckoutYear,CheckoutMonth,Checkouts,Subjects,Publisher,PublicationYear,MaterialType
0,Physical,Horizon,2005,4,1,Tsunamis,NaN,NaN,BOOK
1,Physical,Horizon,2005,4,1,Tsunamis Juvenile literature,NaN,NaN,BOOK
2,Physical,Horizon,2005,4,3,Cinco de Mayo Mexican holiday History Juvenile...,"Enslow Publishers,",c2002.,BOOK
3,Physical,Horizon,2005,4,3,Mexican Americans Social life and customs Juv...,"Enslow Publishers,",c2002.,BOOK
4,Physical,Horizon,2005,4,3,Puebla Battle of Puebla de Zaragoza Mexico 18...,"Enslow Publishers,",c2002.,BOOK
5,Physical,Horizon,2005,4,3,Cinco de Mayo Mexican holiday Juvenile litera...,"Enslow Publishers,",c2002.,BOOK
6,Physical,Horizon,2005,4,3,Holidays Juvenile literature,"Enslow Publishers,",c2002.,BOOK
7,Physical,Horizon,2005,4,3,Mexico Social life and customs Juvenile liter...,"Enslow Publishers,",c2002.,BOOK
8,Physical,Horizon,2005,4,1,War stories,NaN,NaN,BOOK
9,Physical,Horizon,2005,4,1,Historical fiction,NaN,NaN,BOOK


In [0]:
procs = [FillMissing, Categorify, Normalize]

In [0]:
valid_idx = range(len(df)-6000, len(df))

In [0]:
dep_var = 'MaterialType'
cat_names = ['UsageClass', 'CheckoutType',	'CheckoutYear',	'CheckoutMonth',	'Subjects',	'Checkouts', 'Publisher',	'PublicationYear']

In [43]:
data = TabularDataBunch.from_df('/content/drive/My Drive/Colab Notebooks/dataset/hackerearth', df, dep_var, valid_idx=valid_idx, procs=procs, cat_names=cat_names)
print(data.train_ds.cont_names)

[]


In [44]:
(cat_x,cont_x),y = next(iter(data.train_dl))
for o in (cat_x, cont_x, y): print(to_np(o[:5]))

[[    1     1     1     1  4724     2     0     0]
 [    1     1     1     1 16843     1   867   218]
 [    1     1     1     1  7232     1     0     0]
 [    1     1     1     1 13166     2     0     0]
 [    1     1     1     1     0     1     0     0]]
[0 0 0 0 0]
[0 0 0 0 2]


In [45]:
learn = tabular_learner(data, layers=[200,100], emb_szs={'CheckoutType': 10}, metrics=accuracy)
learn.fit_one_cycle(20, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.718480,0.844048,0.792667,00:03
1,0.253860,1.038742,0.781667,00:04
2,0.277555,1.070004,0.786500,00:04
3,0.254084,1.058299,0.791000,00:04
4,0.230442,0.976639,0.788000,00:04
5,0.184604,0.798625,0.799667,00:04
6,0.167200,1.064620,0.795167,00:04
7,0.130395,1.252963,0.534833,00:04
8,0.132400,0.873729,0.799167,00:04
9,0.116133,1.085698,0.796667,00:04


In [46]:
pred_class, pred_idx, outputs = learn.predict(df.iloc[0])
str(pred_class)

'BOOK'

In [0]:
df2 = pd.read_csv('https://raw.githubusercontent.com/Sapna20/ai3/master/test_file.csv')

In [0]:
del df2['ID']
del df2['Title']
del df2['Creator']

In [0]:
df3 = pd.read_csv('https://raw.githubusercontent.com/Sapna20/ai3/master/sample_submission.csv')

In [0]:
j=0
for i in range(21101):
  pred_class, pred_idx, outputs = learn.predict(df2.iloc[i])
  
  df3.loc[j,'MaterialType'] = str(pred_class)
  j=j+1

In [54]:
df3

,ID,MaterialType
0,31654,BOOK
1,31655,VIDEOCASS
2,31656,SOUNDDISC
3,31657,BOOK
4,31658,BOOK
5,31659,BOOK
6,31660,SOUNDDISC
7,31661,BOOK
8,31662,BOOK
9,31663,BOOK


In [0]:
export_csv = df3.to_csv (r'/content/drive/My Drive/Colab Notebooks/dataset/hackerearth/sample_submission5.csv', index = None, header=True) 